<a href="https://colab.research.google.com/github/Chu-Yichen/QM2-Group-19/blob/main/Data_Collection_%26_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Set up and downloading the data**

In [1]:
!pip install requests --quiet

import os
import math
import requests
import zipfile
import pandas as pd

# Pollutants of interest with their parameter codes:
# Ozone (O3 - ppm): 44201
# PM2.5 (µg/m3): 88101
# PM10 (µg/m3): 81102
# CO (ppm): 42101
# SO2 (ppb): 42401
# NO2 (ppb): 42602

pollutants = {
    "O3": 44201,
    "PM2.5": 88101,
    "PM10": 81102,
    "CO": 42101,
    "SO2": 42401,
    "NO2": 42602
}

years = range(1999, 2025)  # The range of years for which data will be downloaded, aligns with the literature review

# Directory to store the downloaded data
data_dir = "epa_data"
os.makedirs(data_dir, exist_ok=True)
base_url = "https://aqs.epa.gov/aqsweb/airdata"

# Download and unzip data files for all pollutants and years
for param_name, param_code in pollutants.items():
    for year in years: # Iterate over pollutants and years
        zip_filename = f"daily_{param_code}_{year}.zip" #Construct the filename for the zipped data
        url = f"{base_url}/{zip_filename}" # Full URL to the file
        local_zip_path = os.path.join(data_dir, zip_filename)# Local path

        if not os.path.isfile(local_zip_path):
            r = requests.get(url)
            if r.status_code == 200:
                with open(local_zip_path, 'wb') as f:
                    f.write(r.content)
                # Unzip the downloaded file into the data directory
                with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
                    zip_ref.extractall(data_dir)
            else:
                # If file doesn't exist for a given year/pollutant, skip
                print(f"File not found: {zip_filename}")
        else:
            # Unzip if not already done
            csv_filename = f"daily_{param_code}_{year}.csv"
            if not os.path.isfile(os.path.join(data_dir, csv_filename)):
                with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
                    zip_ref.extractall(data_dir)


**Identify Sites Near Atlanta Airport (<20km) and Extract Their Info**

In [ ]:
# Load all CSV files and combine into a single DataFrame
all_csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]

# List to store individual DataFrames
df_list = []
for csv_file in all_csv_files:
    csv_path = os.path.join(data_dir, csv_file)
    # Use low_memory=False to avoid potential DtypeWarnings for large files
    temp_df = pd.read_csv(csv_path, low_memory=False)
    df_list.append(temp_df)

if len(df_list) == 0:
    # Raise an error if no data files are found
    raise ValueError("No data files found. Please check that data is available.")

# Combine all DataFrames into one
combined_df = pd.concat(df_list, ignore_index=True)

# Ensure the date column is properly parsed
if "Date Local" not in combined_df.columns:
    # Raise an error if the expected column is missing
    raise ValueError("Could not find 'Date Local' column in the data.")
combined_df["Date Local"] = pd.to_datetime(combined_df["Date Local"], errors='coerce')

# Drop rows without valid dates
combined_df = combined_df.dropna(subset=["Date Local"])

# Coordinates of the Altlanta International Airport (used for proximity calculations)
airport_lat, airport_lon = 33.6407, -84.4277

# Function to calculate Haversine distance between two coordinates
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in kilometers
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Check for latitude and longitude columns
if "Latitude" not in combined_df.columns or "Longitude" not in combined_df.columns:
    raise ValueError("No 'Latitude' or 'Longitude' columns found.")

# Identify unique monitoring sites
site_cols = ["State Code", "County Code", "Site Num", "Latitude", "Longitude"]
unique_sites = combined_df[site_cols].drop_duplicates()

# Calculate the distance of each site from the airport
unique_sites["distance_km"] = unique_sites.apply(
    lambda row: haversine_distance(airport_lat, airport_lon, row['Latitude'], row['Longitude']), axis=1
)

# Filter sites within a 20 km radius of the airport
nearby_sites = unique_sites[unique_sites['distance_km'] < 20].copy()

# Display the results
print("Nearby sites found (<20 km):")
print(nearby_sites)


**Check Data Availability for These Sites**

In [ ]:
# Merge site information back to the combined DataFrame to include only nearby sites
key_cols = ["State Code", "County Code", "Site Num"]
nearby_keys = nearby_sites[key_cols].drop_duplicates()

# Perform an inner join to filter the combined data to include only nearby sites
merged = pd.merge(combined_df, nearby_keys, on=key_cols, how='inner')

# Add a 'Year' column for convenience in grouping data
merged["Year"] = merged["Date Local"].dt.year

# Group data by site, parameter, and year to calculate the number of observations
availability = (merged
                .groupby(["State Code", "County Code", "Site Num", "Parameter Code", "Year"])["Sample Measurement"]
                .count()
                .reset_index(name="Observations"))




**Retrieve and Print Out the Final Filtered Data**

In [ ]:
# Filter by date range
start_date = pd.to_datetime("1999-01-01")
end_date = pd.to_datetime("2024-12-31")

# Filter data to include only rows within the specified date range
filtered = merged[(merged["Date Local"] >= start_date) & (merged["Date Local"] <= end_date)].copy()

# Define a truncate function to truncate values to a specified number of decimal places
def truncate(value, decimals=0):
    # Truncate instead of rounding to match technical requirements
    if pd.isna(value):
        return value
    factor = 10**decimals
    return math.floor(value * factor) / factor

# Mapping of parameter codes to truncation logic
param_truncation = {
    44201: ("O3", lambda x: truncate(x, 3)),   # O3 ppm -> 3 decimals
    88101: ("PM2.5", lambda x: truncate(x, 1)),# PM2.5 µg/m3 -> 1 decimal
    81102: ("PM10", lambda x: truncate(x, 0)), # PM10 µg/m3 -> integer
    42101: ("CO", lambda x: truncate(x, 1)),   # CO ppm -> 1 decimal
    42401: ("SO2", lambda x: truncate(x, 0)),  # SO2 ppb -> integer
    42602: ("NO2", lambda x: truncate(x, 0))   # NO2 ppb -> integer
}

# Add parameter names and apply truncation to measurements
filtered["Parameter Name"] = filtered["Parameter Code"].map(lambda pc: param_truncation[pc][0] if pc in param_truncation else "Unknown")
filtered["Truncated Measurement"] = filtered.apply(
    lambda row: param_truncation[row["Parameter Code"]][1](row["Sample Measurement"]) if row["Parameter Code"] in param_truncation else row["Sample Measurement"],
    axis=1
)

# Select and display relevant columns for final output
final_df = filtered[[
    "Date Local",
    "State Code",
    "County Code",
    "Site Num",
    "Parameter Code",
    "Parameter Name",
    "Latitude",
    "Longitude",
    "Sample Measurement",
    "Truncated Measurement"
]].copy()

# Display the final filtered data
print("Final filtered data:")
print(final_df.head())
